<a href="https://colab.research.google.com/github/MudassirABBASSi/Computer-Vision/blob/main/Yolo_(object_detection_library).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Configure API keys
To fine-tune YOLO11, you need to provide your Roboflow API key. Follow these steps:

Go to your Roboflow Settings page. Click Copy. This will place your private key in the clipboard.
In Colab, go to the left pane and click on Secrets (🔑). Store Roboflow API Key under the name ROBOFLOW_API_KEY.

In [1]:
!nvidia-smi

Tue Mar 11 16:51:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              8W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


# yolo installition

In [ ]:
%pip install "ultralytics<=8.3.40" supervision roboflow
import ultralytics
ultralytics.checks()

#CLI

In [ ]:
!yolo task=detect mode=predict model=yolo11n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [ ]:
from IPython.display import Image as IPyImage
IPyImage(filename=f'{HOME}/runs/detect/predict/dog.jpg', width=600)

In [ ]:
!yolo task=detect mode=predict model=yolo11n.pt conf=0.25 source='/content/ws.jpg' save=True

In [ ]:
from IPython.display import Image as IPyImage
IPyImage(filename=f'{HOME}/runs/detect/predict2/ws.jpg', width=600)

In [ ]:
!yolo task=detect mode=predict model=yolo11n.pt conf=0.25 source='/content/images.jpeg' save=True

In [ ]:
from IPython.display import Image as IPyImage
IPyImage(filename=f'{HOME}/runs/detect/predict3/images.jpg', width=600)

# SDK
NOTE: YOLO's Python interface allows for seamless integration into your Python projects, making it easy to load, run, and process the model's output.

#url uploaded

In [ ]:
from ultralytics import YOLO
from PIL import Image
import requests

model = YOLO('yolo11n.pt')
image = Image.open(requests.get('https://media.roboflow.com/notebooks/examples/dog.jpeg', stream=True).raw)
result = model.predict(image, conf=0.25)[0]
import supervision as sv

detections = sv.Detections.from_ultralytics(result)


box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK)

annotated_image = image.copy()
annotated_image = box_annotator.annotate(annotated_image, detections=detections)
annotated_image = label_annotator.annotate(annotated_image, detections=detections)

sv.plot_image(annotated_image, size=(10, 10))

# upload pic

In [ ]:
import os
from PIL import Image
import requests
from ultralytics import YOLO
import supervision as sv

# Load the pretrained YOLO model
model = YOLO("yolo11m.pt")  # Ensure the model file exists in your working directory

try:
    # Define the image path (can be a local file path or a full URL)
    image_path = '/content/images.jpeg'  # Replace with a full URL if it's remote, e.g., 'https://example.com/ws.jpg'

    # Check if the image_path is a URL (starts with http:// or https://)
    if image_path.startswith("http://") or image_path.startswith("https://"):
        image = Image.open(requests.get(image_path, stream=True).raw)
    else:
        image = Image.open(image_path)

    # Perform object detection and save results
    results = model.predict(source=image, save=True)
    result = results[0]

    # Convert the model output to a format suitable for annotation
    detections = sv.Detections.from_ultralytics(result)

    # Set up annotators for bounding boxes and labels
    box_annotator = sv.BoxAnnotator()
    label_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK)

    # Annotate the image
    annotated_image = image.copy()
    annotated_image = box_annotator.annotate(annotated_image, detections=detections)
    annotated_image = label_annotator.annotate(annotated_image, detections=detections)

    # Display the annotated image
    sv.plot_image(annotated_image, size=(5, 10))

except Exception as e:
    print(f"An error occurred: {e}")


# accapting the image from the user personal computer

In [ ]:
import io
from PIL import Image
from ultralytics import YOLO
import supervision as sv
import ipywidgets as widgets
from IPython.display import display

def run_detection(image):
    # Load the pretrained YOLO model (ensure "yolo11m.pt" is in your working directory)
    model = YOLO("yolo11m.pt")

    try:
        # Run object detection on the provided PIL Image
        results = model.predict(source=image, save=True)
        result = results[0]

        # Convert model output to a format suitable for annotation
        detections = sv.Detections.from_ultralytics(result)

        # Annotate the image with bounding boxes and labels
        box_annotator = sv.BoxAnnotator()
        label_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK)
        annotated_image = image.copy()
        annotated_image = box_annotator.annotate(annotated_image, detections=detections)
        annotated_image = label_annotator.annotate(annotated_image, detections=detections)

        # Display the annotated image
        sv.plot_image(annotated_image, size=(5, 10))

    except Exception as e:
        print(f"Error during detection: {e}")

# Create a file upload widget that accepts image files
upload_widget = widgets.FileUpload(accept='image/*', multiple=False)
display(upload_widget)

def on_upload_change(change):
    if upload_widget.value:
        # Get the first uploaded file's content as bytes
        uploaded_file = next(iter(upload_widget.value.values()))
        content = uploaded_file['content']
        # Convert bytes data to a PIL Image
        image = Image.open(io.BytesIO(content))
        # Run detection on the uploaded image
        run_detection(image)

# Observe changes to the upload widget's value
upload_widget.observe(on_upload_change, names='value')


# fine tunning

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

workspace = rf.workspace("liangdianzhong")
project = workspace.project("-qvdww")
version = project.version(3)
dataset = version.download("yolov11")

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=10 imgsz=640 plots=True

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)